In [1]:
import pandas as pd
from glob import glob
from work import *
from tqdm import tqdm
tqdm.pandas()

c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [2]:
path = "../data/우리은행/작업/공통/dataset_16.csv"
df = pd.read_csv(path, encoding="utf-8-sig", index_col=0)
df.reset_index(drop=True, inplace=True)


# df_1 = df[(df["완료여부"]==1)].copy()
df_1 = df[(df["완료여부"]==1) & (df["분류"]=="급여지급")].copy()

use_index = df_1.index
df_2 = df[~df.index.isin(use_index)].copy()




c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (4,16,18,19,21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
# for i in df["분류"].unique():
#     etc = df[df["분류"]==i].copy()
#     i = re.sub(r"[^\w\s]", " ", i) # 특수문자 제거
#     etc.to_csv("../data/우리은행/작업/분류별/%s.csv"%i, encoding="utf-8-sig", index=False)

In [147]:
def dele_spec_str(string):
    try:
        # string = string.replace(" ", "")
        # string = string.replace("　", "")
        string = string.replace("-", " ")
        string = string.replace("－", " ")
        return string
    except AttributeError as e:
        print(e, string)
        return ""

# df_1["단어"] = df_1["단어"].progress_apply(lambda x: dele_spec_str(x))
# df_1["단어의미"] = df_1["단어의미"].progress_apply(lambda x: dele_spec_str(x))
# df_1["적요 설명"] = df_1["적요 설명"].progress_apply(lambda x: dele_spec_str(x))

def change_NAME_word(string, word):
    if word == "#NAME?":
        string = dele_spec_str(string)
        return string.split()[1]
    else:
        return word

def change_NAME_mean(string, textmean):
    if textmean == "#NAME?":
        string = dele_spec_str(string)
        return string.split()[1]+"에 기부한 내역의 적요이다."
    else:
        return textmean
    
# df_1["단어"] = df_1.progress_apply(lambda x: change_NAME_word(x["적요"], x["단어"]), axis=1)
# df_1["단어의미"] = df_1.progress_apply(lambda x: change_NAME_word(x["적요"], x["단어의미"]), axis=1)
# df_1["적요 설명"] = df_1.progress_apply(lambda x: change_NAME_mean(x["적요"], x["적요 설명"]), axis=1)


def mod_pay(string):
    string = string.replace("월급여로부터", "으로부터")
    string = string.replace("급여로부터", "으로부터")
    if string.startswith('급여'):
        string = string[2:]
    
    return string

def mod_pay_2(string):
    string = string.replace("로부터", "으로부터")
    string = string.replace("으으로부터", "으로부터")    
    return string


def mod_pay_3(string):
    wrong_text = ["이체으로부터", "지급으로부터", "으로부터", "수당으로부터", "정기으로부터", "정기급여", "정기상여", "정산급여", "정립급여", "추가으로부터"]
    for text in wrong_text:
        if string.startswith(text):
            string = "급여를 입금받은 내역의 적요이다."
    if string.startswith('월급여'):
        string = string[3:] 
    return string

# df_1["적요 설명"] = df_1.progress_apply(lambda x: mod_pay_3(x["적요 설명"]), axis=1)

def mod_tex(string, text_mean):
    wrong_text = ["세무법인", "세무회계", "회계법인"]
    for text in wrong_text:
        if string.startswith(text) or string.endswith(text):
             text_mean = text+"으로 이체한 내역의 적요이다."
             break
    return text_mean

def mod_tex_2(string, text_mean):
    wrong_text = ["세무사"]
    for text in wrong_text:
        if string.endswith(text):
             text_mean = text+"에게 이체한 내역의 적요이다."
             break
    return text_mean

def mod_tex_3(string, word):
    wrong_text = ["세무사"]
    for text in wrong_text:
        if string.endswith(text):
            if word == '세무':
                word = "세무사"
            else:
                # 단어
                # word = word[:-1]
                # 단어의미
                word = "이름"
    return word



# df_1["적요 설명"] = df_1.progress_apply(lambda x: mod_tex_2(x["적요"], x["적요 설명"]), axis=1)
# df_1["단어"] = df_1.progress_apply(lambda x: mod_tex_3(x["적요"], x["단어"]), axis=1)
# df_1["단어의미"] = df_1.progress_apply(lambda x: mod_tex_3(x["적요"], x["단어의미"]), axis=1)

def change_ga(string):
    first_string = string.split()[0]
    if first_string.endswith("가"):
        first_string = first_string[:-1] + "에"
    new_string = list()
    new_string.append(first_string)
    for word in string.split()[1:]:
        new_string.append(word)
    return " ".join(new_string)
    
# df_1["적요 설명"] = df_1.progress_apply(lambda x: change_ga(x["적요 설명"]), axis=1)


def change_ga(string):
    first_string = string.split()[0]
    if first_string.endswith("가"):
        first_string = first_string[:-1] + "에"
    new_string = list()
    new_string.append(first_string)
    for word in string.split()[1:]:
        new_string.append(word)
    return " ".join(new_string)

def mod_pay(string):
    return string[:2]+"은행에 급여로 이체한 내역의 적요이다."




# df_1["적요 설명"] = df_1.progress_apply(lambda x: mod_pay(x["적요 설명"]), axis=1)


100%|██████████| 438/438 [00:00<00:00, 108949.42it/s]


In [148]:
df = pd.concat([df_1, df_2], axis=0)

path = "../data/우리은행/작업/공통/dataset_16.csv"
df.to_csv(path, encoding="utf-8-sig")

In [60]:

change_NAME_mean("신한-연화원")

'연화원에 기부한 내역의 적요이다.'

''